# Yolo V5 training

In [1]:
# Train YOLOv5s on COCO128 for 3 epochs
%cd /home/saidinesh/Desktop/Projects/yolov5/
!python train.py --img 640 --cfg models/yolo-tph.yaml --batch 2 --epochs 100 --data datasets/dataset.yaml --weights yolov5s.pt --cache --name "yolov5s-tph"

/home/saidinesh/Desktop/Projects/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: saidineshpola. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=models/yolo-tph.yaml, data=datasets/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s-tph, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ul

# predictions exploration

In [3]:
import torch
import torchvision

# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('ultralytics/yolov5', 'custom', 'runs/train/baseline-yolov5s/weights/last.pt')
model.to(device)
model.eval()

Using cache found in /home/saidinesh/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-19 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7982MiB)

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
!python detect.py --weights runs/train/baseline-yolov5s/weights/last.pt \
    --data datasets/dataset.yaml \
    --source datasets/val/images --save-txt --save-conf \
    --img 640 --half

In [1]:
import fiftyone as fo
name = "my-dataset"
dataset_dir = "datasets/"

# The splits to load
splits = ["train", "val"]
try:
    dataset = fo.load_dataset(name)
    dataset.delete()
except:
    pass
dataset = fo.Dataset(name)    
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

# Get some summary information about the dataset
print(dataset.info)
print(dataset.stats)
session = fo.Session(dataset=dataset)

 100% |█████████████████████| 0/0 [2.7ms elapsed, ? remaining, ? samples/s]  
 100% |█████████████████████| 0/0 [2.4ms elapsed, ? remaining, ? samples/s]  
{}
<bound method Dataset.stats of Name:        my-dataset
Media type:  None
Num samples: 0
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)>



Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 s

In [5]:
import fiftyone as fo
from PIL import Image
from torchvision.transforms import functional as func

name = "my-dataset"
dataset_dir = "datasets/"

# The splits to load
splits = ["train", "val"]
try:
    dataset = fo.load_dataset(name)
    dataset.delete()
except:
    pass
dataset = fo.Dataset(name)    
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

# Get some summary information about the dataset
print('datasetInfo')
print(dataset.info)
# print(dataset.stats)
#session = fo.Session(dataset=dataset)



# Get class list
classes = dataset.default_classes
test_view = dataset.match_tags("val")
# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(test_view):
        # Load image
        image = Image.open(sample.filepath)

        # Perform inference
        preds = model(image)
        pd  = preds.pandas().xyxy[0]

        image = func.to_tensor(image).to(device)
        c, h, w = image.shape

        detections = []

        for i in pd.values: 
            x1, y1, x2, y2 = i[0],i[1],i[2],i[3]
            rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

            detections.append(
                fo.Detection(
                    label=classes[i[5]],
                    bounding_box=rel_box,
                    confidence=i[4]
                )
            )

        # Save predictions to dataset
        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

print("Finished adding predictions to the test_view")

 100% |█████████████████████| 0/0 [46.9us elapsed, ? remaining, ? samples/s]  100% |█████████████████████| 0/0 [2.5ms elapsed, ? remaining, ? samples/s]  
 100% |█████████████████████| 0/0 [3.1ms elapsed, ? remaining, ? samples/s]  
datasetInfo
{}
 100% |█████████████████████| 0/0 [5.1ms elapsed, ? remaining, ? samples/s] 
Finished adding predictions to the test_view


In [12]:
test_view.info

{}

In [1]:
# Training CV model with kfold

In [1]:
%cd /home/saidinesh/Desktop/Projects/yolov5/
from ultralytics import YOLO
weights_path = 'runs/detect/yolo-v5s-base_fold_4/weights/last.pt'
results = {}
for k in range(4,5):
    model = YOLO(weights_path, task='detect')
    dataset_yaml = f'datasets/train/2023-08-30_5-Fold_Cross-val/split_{k+1}/split_{k+1}_dataset.yaml' #ds_yamls[k]
    model.train(data=dataset_yaml,name = f'yolo-v5s-base_fold_{k}',device=0,batch=8,resume=True)  # Include any training arguments
    results[k] = model.metrics  # save output metrics for further analysis
    del model

/home/saidinesh/Desktop/Projects/yolov5


New https://pypi.org/project/ultralytics/8.0.167 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.158 🚀 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7982MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/yolo-v5s-base_fold_4/weights/last.pt, data=datasets/train/2023-08-30_5-Fold_Cross-val/split_5/split_5_dataset.yaml, epochs=100, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=yolo-v5s-base_fold_4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, 


                   from  n    params  module                                       arguments                     
  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [ ]:
print('Completed Successfully')

In [ ]:
import os
import shutil

# Define the paths to the val and train folders
val_folder = '/home/saidinesh/Desktop/Projects/yolov5/datasets/val/images'
train_folder = '/home/saidinesh/Desktop/Projects/yolov5/datasets/train/images'

# Get a list of filenames in both folders
val_files = os.listdir(val_folder)
train_files = os.listdir(train_folder)

# Find common filenames between the two folders
common_files = set(val_files) & set(train_files)

# Move common files from train to val folder
for filename in common_files:
    src_path = os.path.join(train_folder, filename)
    dest_path = os.path.join(val_folder, filename)
    
    # Check if the file already exists in the val folder
    if os.path.exists(dest_path):
        shutil.move(src_path, dest_path)
        print(f"Moved '{filename}' from train to val folder.")
        

print("Done.")
